# Test qantitation model performance

## identity model

In [1]:
import torch
import os
import datetime
import pandas as pd
from  torch.utils.data import Dataset
from skimage import io, transform
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import json
import torch
import torchvision.models as models
# from torchsummary import summary
starttime = datetime.datetime.now()
device = 'cpu'
# model = torch.jit.load('resnet/quantization/resnet18.pth').to(device)
# model = torch.jit.load('resnet/quantization/resnext101_32x8d.pth').to(device)
model = torch.jit.load('../../quantization_models/mobilenet_v2.pth').to(device)
# model = torch.jit.load('quantization_models/inception_v3.pth').to(device)
# model_name = 'inception_v3'
# model = models.quantization.__dict__[model_name](pretrained=True,progress=True,quantize=True).to(device)

criterion = nn.CrossEntropyLoss()#thenn.Softmax(dim=1) calculating the Loss
model.eval()

RecursiveScriptModule(
  original_name=QuantizableMobileNetV2
  (features): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=ConvBNReLU
      (0): RecursiveScriptModule(original_name=ConvReLU2d)
      (1): RecursiveScriptModule(original_name=Identity)
      (2): RecursiveScriptModule(original_name=Identity)
    )
    (1): RecursiveScriptModule(
      original_name=QuantizableInvertedResidual
      (conv): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=ConvBNReLU
          (0): RecursiveScriptModule(original_name=ConvReLU2d)
          (1): RecursiveScriptModule(original_name=Identity)
          (2): RecursiveScriptModule(original_name=Identity)
        )
        (1): RecursiveScriptModule(original_name=Conv2d)
        (2): RecursiveScriptModule(original_name=Identity)
      )
      (skip_add): RecursiveScriptModule(original_name=QFunctional)
    )
    (2): Recursiv

In [2]:
root_dir = "../../my_imagenet/ImageNet_validset/"#identify the classification 
class_label_location = "../../my_imagenet/scrip/imagenet_class_index.txt"
class_names = []
with open(class_label_location) as f:
    lines=f.readlines()
    class_to_lable_json = lines[0]
    class_to_lable_dict = json.loads(s=class_to_lable_json)
    for key in class_to_lable_dict:
        class_names.append(class_to_lable_dict[key][0])
        
def get_class_from_path(value):
    for key in class_to_lable_dict:
        if value == class_to_lable_dict[key][0]:
            return int(key)

In [3]:
def get_meta(root_dir, dirs):
    """ Fetches the meta data for all the images and assigns labels.
    """
    paths, classes = [], []
    for i, dir_ in enumerate(dirs):
        for entry in os.scandir(root_dir + dir_):
            if (entry.is_file()):
                paths.append(entry.path)
                classes.append(get_class_from_path(dir_))
                
    return paths, classes

In [4]:
# Benign images we will assign class 0, and malignant as 1
paths, classes = get_meta(root_dir, class_names)

data = {
    'path': paths,
    'class': classes
}

data_df = pd.DataFrame(data, columns=['path', 'class'])
data_df = data_df.sample(frac=1).reset_index(drop=True) # Shuffles the data

In [5]:
batch_size = 32
log_time = 100
validset_size = len(data_df)
print(len(data_df))
print(data_df.head())

25777
                                                path  class
0  my_imagenet/ImageNet_validset/n01498041\ILSVRC...      6
1  my_imagenet/ImageNet_validset/n02992211\ILSVRC...    486
2  my_imagenet/ImageNet_validset/n02389026\ILSVRC...    339
3  my_imagenet/ImageNet_validset/n04120489\ILSVRC...    770
4  my_imagenet/ImageNet_validset/n02132136\ILSVRC...    294


In [6]:
class ImageNet10(Dataset):
    """ ImageNet10 dataset. """

    def __init__(self, df, transform=None):
        """
        Args:
            image_dir (string): Directory with all the images
            df (DataFrame object): Dataframe containing the images, paths and classes
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load image from path and get label
        x = Image.open(self.df['path'][index])
        try:
          x = x.convert('RGB') # To deal with some grayscale images in the data
        except:
          pass
        y = torch.tensor(int(self.df['class'][index]))

        if self.transform:
            x = self.transform(x)

        return x, y

In [7]:
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

ins_dataset_valid = ImageNet10(
#     df=data_df[0:int(len(data_df)/10)],
    df=data_df,
    transform=data_transform,
)
valid_dataset_loader = torch.utils.data.DataLoader(
    ins_dataset_valid,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)


In [8]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [9]:
def single_valid_dataset_gpu():
    print('=====================================================================')
    running_loss = 0.0
    correct = 0
    total = 0
    acc1 = []
    acc5 = []
    with torch.no_grad():  
        for i, data in enumerate(valid_dataset_loader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            if i == 0 :
                starttime_each_log = datetime.datetime.now()
                print('Model inference time: {}'.format(starttime_each_log - starttime))
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            #             _, predicted = torch.max(outputs.data, 1)
            #             total += labels.size(0)
            #             total += len(labels)
            #             correct += (predicted == labels).sum().item()
            acc1_temp, acc5_temp = accuracy(outputs, labels, topk=(1, 5))
#             softmax_output = torch.nn.functional.softmax(outputs, dim=0)
#             acc1_temp, acc5_temp = accuracy(softmax_output, labels, topk=(1, 5))
            acc1.append(acc1_temp)
            acc5.append(acc5_temp)
            #             if i%(validset_size/batch_size/log_time) == 0 and i != 0:
            if i%log_time == 0 and i != 0:
                endtime_each_log = datetime.datetime.now()
                time_comsumption = endtime_each_log - starttime_each_log
                starttime_each_log = endtime_each_log
                print('validset    Loss: {:.4f}   Top1 Accuracy : {:.4f}%  Top5 accuracy : {:.4f}%  Time comsumption:{}'.format(running_loss/(i*batch_size), acc1_temp.item(), acc5_temp.item(), time_comsumption)) 
            #                 print('validset    Loss: {:.4f}   Accuracy : {:.4f}%   Time comsumption:{}'.format(running_loss/(i*batch_size), 100 * correct / total, time_comsumption)) 
    print("===========================================================================")
    acc1_sum = 0
    for acc1_temp in acc1:
        acc1_sum += acc1_temp
    acc1_average = acc1_sum/len(acc1)
    acc5_sum = 0
    for acc5_temp in acc5:
        acc5_sum += acc5_temp
    acc5_average = acc5_sum/len(acc5)
    print('overall validset    Loss: {:.4f}   Top1 Accuracy : {:.4f}%  Top5 accuracy : {:.4f}% Time comsumption:{}'.format(running_loss/(i*batch_size), acc1_average.item(), acc5_average.item(), (datetime.datetime.now() - starttime)))

In [10]:
def get_parameter_number(net):
    total_num = sum(p.numel() for p in net.parameters())
    trainable_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}
print(get_parameter_number(model))

{'Total': 0, 'Trainable': 0}


## Testing

In [11]:
# summary(model, (3, 256, 256))
single_valid_dataset_gpu()

BrokenPipeError: [Errno 32] Broken pipe

In [12]:
# print(model.parameters)
params = model.
print(params)
k = 0
for i in params:
    l = 1
    print("该层的结构：" + str(list(i.size())))
    for j in i.size():
        l *= j
    print("该层参数和：" + str(l))
    k = k + l
print("总参数数量和：" + str(k))

SyntaxError: invalid syntax (<ipython-input-12-a2b83d60b80a>, line 2)

In [13]:
def single_valid_dataset_gpu():
    starttime = datetime.datetime.now()
    starttime_each_log = starttime
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  
         for i, data in enumerate(valid_dataset_loader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
#             total += len(labels)
            correct += (predicted == labels).sum().item()

#             if i%(validset_size/batch_size/log_time) == 0 and i != 0:
            if i%log_time == 0 and i != 0:
                endtime_each_log = datetime.datetime.now()
                time_comsumption = endtime_each_log - starttime_each_log
                starttime_each_log = endtime_each_log
                print('validset    Loss: {:.4f}   Accuracy : {:.4f}%   Time comsumption:{}'.format(running_loss/(i*batch_size), 100 * correct / total, time_comsumption)) 
    print("===========================================================================")
    print('overall validset    Loss: {:.4f}   Accuracy : {:.4f}%   Time comsumption:{}'.format(running_loss/(i*batch_size), 100 * correct / total, (datetime.datetime.now() - starttime))) 


In [2]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0).to(device) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
# if torch.cuda.is_available():
#     input_batch = input_batch.to('cuda')
#     model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
_,predicted = torch.max(output.data, 1)

print(output[0])
print(predicted)
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.

tensor([ 1.8372,  0.4593, -5.3585, -4.4399, -4.2868,  0.3062, -1.2248,  2.2965,
         5.6647, -3.9806, -6.1240, -4.7461, -4.2868, -4.2868, -3.0620, -2.7558,
        -2.7558, -0.9186, -2.6027, -5.6647, -5.2054, -1.9903, -5.5116,  1.3779,
        -0.6124, -2.7558, -1.5310, -0.1531, -2.7558,  2.2965,  0.1531,  1.6841,
         0.4593, -7.1957, -4.4399, -3.6744,  0.9186,  0.1531,  1.5310, -1.2248,
         1.9903, -1.9903, -4.1337, -1.5310,  0.4593, -2.6027,  3.6744,  0.7655,
        -0.7655, -0.9186,  0.4593, -0.4593,  0.1531, -2.2965,  1.0717, -2.2965,
        -4.7461, -5.6647, -3.0620, -1.5310,  0.4593, -4.1337, -2.4496, -5.3585,
        -3.3682, -6.5833,  0.4593, -1.5310, -0.4593, -1.6841, -3.0620, -2.7558,
        -2.2965, -2.6027, -1.8372, -1.3779, -2.4496, -1.9903,  0.3062,  1.6841,
        -2.2965, -1.2248,  3.9806,  2.1434,  1.8372, -1.3779,  0.4593,  0.6124,
        -2.6027,  4.1337, -0.4593, -6.5833, -5.6647, -4.7461, -0.4593, -6.8895,
        -0.9186, -2.2965, -7.0426,  2.90